In [1]:
import pandas as pd
import numpy as np
# import guidedlda
import time

from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import string

"""BQ imports"""
from google.cloud import bigquery
client = bigquery.Client()

from itertools import chain
from collections import Counter

In [8]:
!python -m spacy download en_core_web_lg

     |████████████████████████▎       | 594.0 MB 205.6 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 782.7 MB 1.4 MB/s eta 0:00:01
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.3.1-py3-none-any.whl size=782936122 sha256=636e499a4975538320c176783bf75e01b692556d66a32082b25abc4fff1bfe13
  Stored in directory: /tmp/pip-ephem-wheel-cache-cybpgaho/wheels/41/75/77/c4a98e18b2c317a2a13931cbbea7e3ca7f3a21efc36adc1d71
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [19]:
!pip install spacy==2.3.3

     |████████████████████████████████| 10.4 MB 5.0 MB/s eta 0:00:01
  Using cached cymem-2.0.5-cp37-cp37m-manylinux2014_x86_64.whl (35 kB)
     |████████████████████████████████| 184 kB 133.8 MB/s eta 0:00:01
  Using cached blis-0.7.4-cp37-cp37m-manylinux2014_x86_64.whl (9.8 MB)
     |████████████████████████████████| 1.0 MB 100.8 MB/s eta 0:00:01
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached preshed-3.0.5-cp37-cp37m-manylinux2014_x86_64.whl (126 kB)


In [10]:
!pip install pytextrank==2.0.3

     |████████████████████████████████| 242 kB 4.9 MB/s eta 0:00:01


In [16]:
!pip install fuzzywuzzy==0.18.0

In [18]:
!pip install python-Levenshtein==0.12.1

     |████████████████████████████████| 50 kB 3.3 MB/s eta 0:00:011
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.1-cp37-cp37m-linux_x86_64.whl size=178235 sha256=e2474be3e886959b65b0dafd792211df71a347d51762888ddca2f13132d8e398
  Stored in directory: /home/jupyter/.cache/pip/wheels/20/d6/58/279b0443284b2e509e83f6cde4cdb270e2df46db3692c6e667
Successfully built python-Levenshtein


In [2]:
train = pd.read_pickle("./data/preprocess_mnet_train.pkl")

train.drop_duplicates('natid', keep='first', inplace=True)
train.reset_index(drop=True, inplace=True)

print(train.shape)
train.head()

(10078, 8)


,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7
1,blogandpostid/blog/post/4587-60d354bc9690ca000...,2021-06-23,Food and Drink,Travel,Travel Type,Celebrate The Lodge At Woodloch’s 15-Year Anni...,celebrate lodge woodloch's 15-year anniversary...,23
2,blogandpostid/blog/post/3113-5f526f97559f59000...,2020-09-04,Music and Audio,Music and Audio,Hip Hop Music,Cardi B And Megan Thee Stallion Both Earn Thei...,cardi b megan thee stallion earn no. 1 hit you...,12
3,blogandpostid/blog/post/5259-6015f002c7824f000...,2021-02-09,Travel,Shopping,Gifts and Greetings Cards,Valentine’s Day Gift Guide: The Most Romantic ...,valentine's day gift guide: romantic hotel pac...,13
4,blogandpostid/blog/post/4983-5f8064d69a4142000...,2020-10-14,Shopping,Shopping,Sales and Promotions,The Best Prime Day Bike Deals For Saving Big o...,best prime day bike deals saving big bicycles ...,13


In [13]:
print(sorted(train[train.tier1=='Business and Finance'].tier2.unique()))

['Business', 'Economy', 'Industries']


In [3]:
test = pd.read_pickle("./data/preprocess_mnet_test.pkl")
test.drop_duplicates('natid', keep='first', inplace=True)
test.reset_index(drop=True, inplace=True)

print(test.shape)
test.head()

(1258, 8)


,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/3541-60eb91d76f189c000...,2021-07-12,Movies,Movies,Indie and Arthouse Movies,What Will The Movie Industry Look Like After C...,movie industry look like covid?. movie industr...,2
1,blogandpostid/blog/post/7380-5f7a952000e768000...,2020-10-05,Video Gaming,Video Gaming,Video Game Genres,A YouTuber Made An ‘Among Us’ Mode In ‘Overwat...,youtuber us' mode overwatch' looks fantastic. ...,10
2,blogandpostid/blog/post/50373-5fcfe6475a1f9400...,2020-12-08,News and Politics,News and Politics,Politics,Rudy Giuliani Says He’s ‘Doing Fine’ With Covi...,"rudy giuliani says fine' covid, hopes released...",0
3,blogandpostid/blog/post/25-607d3d189d9fd30006d...,2021-04-19,Technology and Computing,Personal Finance,Personal Investing,Will Apple Stock’s Q2 Results Beat Consensus?....,apple stock's q2 results beat consensus?. appl...,21
4,blogandpostid/blog/post/6937-60b6f3feff8c53000...,2021-06-02,Movies,Movies,Documentary Movies,Ava DuVernay’s Array Film Collective Partners ...,ava duvernay's array film collective partners ...,2


In [4]:
full = pd.concat([train, test])
print("Before", full.shape)

full.drop_duplicates('natid', keep='first', inplace=True)
full.reset_index(drop=True, inplace=True)

print("After", full.shape)

Before (11336, 8)
After (11046, 8)


* Get articles which have keyphrases

In [5]:
sql= """
    SELECT 
        * 
    FROM 
        `api-project-901373404215.keyphrase_extraction.forbes_keyphrases` 
    """

# Send the query to the api and return a df
keyphrase_df = client.query(sql).to_dataframe()

print("Shape: ", keyphrase_df.shape) 

Shape:  (77287, 4)


In [11]:
# Checking to make sure we are not processing any articles already in the forbes_keyphrases table

# df = full[~full.natid.isin(keyphrase_df.naturalid)]
# df.reset_index(drop=True, inplace=True)
# df.shape

# 3 got left out somehow. processing them
df = train[~train.natid.isin(all_kps.natid)]
df.reset_index(drop=True, inplace=True)
print(df.shape)
df

(3, 8)


,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/50276-60f734ce80505100...,2021-07-20,News and Politics,Sports,Golf,Trump Says Cheney Challengers Will Vie For His...,trump says cheney challengers vie endorsement ...,18
1,blogandpostid/blog/post/50769-6120124cfd4e7600...,2021-08-20,News and Politics,Family and Relationships,Parenting,Baby Passed To U.S. Troops Over Kabul Airport ...,baby passed usa troops kabul airport fence reu...,11
2,blogandpostid/blog/post/4983-5fad72b4d076a5000...,2020-11-27,Shopping,Shopping,Sales and Promotions,The Best Black Friday Couch Deals: Top Furnitu...,best black friday couch deals: furniture sites...,13


In [12]:
import re
import html.parser
import unicodedata
import spacy
import pytextrank 

STOP_WORDS = pd.read_pickle('./data/forbes_stop_words.pkl')

# Contraction dictionary

contraction_dict_file = './data/contraction_dictionary.pkl'
c_dict = pd.read_pickle(contraction_dict_file)
# Compiling the contraction dict
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

def listToString(s):  
    str1 = " " 
    return (str1.join(s)) 

# Function to expand contractions
def expandContractions(text):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

# Function to remove unicode
def normalize_unicode(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str) # replaces accents by converting to byte literal and then decoding to utf-8.
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode("utf-8")  

# Function to remove the html
class HTMLTextExtractor(html.parser.HTMLParser):
    def __init__(self):
        super(HTMLTextExtractor, self).__init__()
        self.result = []
    def handle_data(self, d):
        self.result.append(d)
    def get_text(self):
        return ''.join(self.result)

def html_to_text(html):
    s = HTMLTextExtractor()
    s.feed(html)
    return s.get_text()

def remove_html(text):
    text = str(text)
    string = re.sub('<[^<]+?>', ' ', text)
    string = html_to_text(string)
    string = re.sub('’', "'", string) 
    string = re.sub(r'[^\x00-\x7F]+', ' ', string) # [^\x00-\x7F]+ searches for all non-ASCII values
    string = re.sub(r'\[.*?\]', ' ', string) # escape special characters mentioned inside []
    string = string.replace('\r', '  ')
    string = string.replace('\n', '  ')
    string = re.sub(' +', ' ', string)
    return string

# Function to remove newline character
def remove_newline(string):
    string = string.replace('\r', '') #remove carriage return
    return string.replace('\n', '') #remove newline


# -------------------------------- SPACY TEXTRANK --------------------------------
try:
    nlp = spacy.load('en_core_web_lg')
except OSError as e:
    print(e)
    print('Downloading spacy en_core_web_lg language model')
    from spacy.cli import download
    download('en_core_web_lg')
    nlp = spacy.load('en_core_web_lg')
    
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name='textrank', last=True)

def spacy_textrank(text):
    num_words = int(len(word_tokenize(text)))
    if(num_words <= 100):
        num_keywords = int(num_words * 0.2)
    elif (num_words > 100 and num_words <= 400):
        num_keywords =  int(num_words * 0.1)
    elif (num_words > 400 and num_words <= 1000):
        num_keywords =  int(num_words * 0.04)
    elif (num_words > 1000 and num_words <= 3000):
        num_keywords =  min(int(num_words * 0.02), 40)
    else:
        num_keywords = min(int(num_words * 0.01), 40)
        
    spacy_textrank = []
    doc = nlp(text)
    for p in doc._.phrases[:num_keywords]:
        spacy_textrank.append(p.text)

    return spacy_textrank

* Pre-process article text

In [13]:
df

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/50276-60f734ce80505100...,2021-07-20,News and Politics,Sports,Golf,Trump Says Cheney Challengers Will Vie For His...,trump says cheney challengers vie endorsement ...,18
1,blogandpostid/blog/post/50769-6120124cfd4e7600...,2021-08-20,News and Politics,Family and Relationships,Parenting,Baby Passed To U.S. Troops Over Kabul Airport ...,baby passed usa troops kabul airport fence reu...,11
2,blogandpostid/blog/post/4983-5fad72b4d076a5000...,2020-11-27,Shopping,Shopping,Sales and Promotions,The Best Black Friday Couch Deals: Top Furnitu...,best black friday couch deals: furniture sites...,13


In [14]:
# Remove html, normalize unicode
df['clean_body'] = df['summary'].astype(str)
df['clean_body'] = df['clean_body'].apply(remove_newline).astype(str)
df['clean_body'] = df['clean_body'].apply(expandContractions)
df['clean_body'] = df['clean_body'].apply(normalize_unicode)
df['clean_body'] = df['clean_body'].apply(remove_html).str.strip()
df['clean_body'] = df['clean_body'].apply(lambda x: re.sub(r"<[^>]*>", " ", x)) 

# Proprocess and clean
df['clean_body'] = df['clean_body'].str.lower()
df['clean_body'] = df['clean_body'].apply(lambda x: re.sub(r'\d+', '', x)) # remove numbers
df['clean_body'] = df['clean_body'].str.replace('%','') # remove '%' sign
df['clean_body'] = df['clean_body'].str.replace('covid- ','covid-19') # patch-fix an important word
df['clean_body'] =  df['clean_body'].replace('\s+', ' ', regex=True) # remove more than 1 space in between words

start = time.time()
df["spacytextrank_keywords"] = df["clean_body"].apply(lambda x: spacy_textrank(x)) 
end = time.time()
print("Initial keyphrases generation time in mins: ", (end - start)/60) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Initial keyphrases generation time in mins:  0.006176706155141195


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
from collections import Counter
from collections import OrderedDict
from fuzzywuzzy import process, fuzz

# ---------Clean keywords---------

# read ner_forbes table
sql = """
    SELECT
        distinct(token), tag 
    FROM `api-project-901373404215.ner.ner_forbes` 
    ORDER BY token 
    """
# Send the query to the api and return a df
google_ner = client.query(sql).to_dataframe()
google_ner = google_ner[google_ner["token"].apply(lambda x: len(x)>1)]

start = time.time()
nlp = spacy.load('en_core_web_lg')
clean_keys = []
for index, row in df.iterrows(): 
    # remove pattern STOP WORDS
    pattern = re.compile("\\b(the|only|other|their|these|our|her|his|such|some|many|most|my|his|all|several|people|earlier|an|your|its|enough|nice|this|$|more|how|how much|topline|further reading|key facts|key background|crucial quote|yesterdays|related reading|morning|evening)\\W", re.I)
    inter_list = [pattern.sub("", x) for x in row["spacytextrank_keywords"]]
    # remove starting "a"
    pattern_a = re.compile(r'^a\s', re.I)
    inter_list = [pattern_a.sub("", x) for x in inter_list]
    # remove STOP WORDS
    inter_list = [x for x in inter_list if x not in STOP_WORDS]
    
    print("after stopw", inter_list)

    # remove NERs from ner_forbes
    inter_list = [x for x in inter_list if x not in list(google_ner.token)]
    
    print("after google ner", inter_list)

    # remove SPACY NER words
    ner_words_to_remove = []
    for elem in inter_list:
        doc1 = nlp(elem)
        for entity in doc1.ents:
            # keeping 'ORG' out for now
            if entity.label_ in ('GPE', 'LOC', 'FAC', 'PERSON',  'DATE'):    
                ner_words_to_remove.append(elem)

    inter_list = [x for x in inter_list if x not in ner_words_to_remove]
    
    print("after spacy ner", inter_list)

    # remove subset-based similar words - e.g. 'pcr tests' and 'tests' then 'tests' will be removed
    inter_list = [i for i in inter_list if not any(i in x and i!=x for x in inter_list)]
    if(len(inter_list)>15):
        # remove similar words using fuzzy match with score>85 - e.g. 'student loans' and 'student loan debt'
        matches_to_remove = []
        for i in range(len(inter_list)):
            process_match = process.extract(inter_list[i], inter_list, scorer=fuzz.partial_ratio) # match every word with other word
            filter_match = [x for x in process_match if x[1]>85 and x[1]<100] 
            matches_to_remove.append([x[0] for x in filter_match])
        matches_to_remove_list = list(set(list(chain(*matches_to_remove))))
        inter_list = [x for x in inter_list if x not in matches_to_remove_list]
    
    print("after similar words", inter_list)
    # remove more than 1 space between words
    inter_list = [re.sub(' +', ' ', x) for x in inter_list]
    # remove extra whitespace
    inter_list = [x.strip() for x in inter_list]
    # remove empty or 1-letter words
    inter_list = [x for x in inter_list if len(x)>1]
    # de-duplicate
    inter_list = list(OrderedDict.fromkeys(inter_list))
    clean_keys.append(inter_list)

end = time.time()
print("Keyword cleaning time in mins: ", (end - start)/60) 

# convert keyword lists to strings for saving in BigQuery table
df['keyphrases'] = clean_keys #6.27

after stopw ['anti-trump rep. liz cheney', 'cheney challengers', 'rep. bennie thompson', 'rep. jamie raskin', 'cheney', 'liz cheney', 'wyoming state sen. anthony bouchard', 'cheney', 'donald trump', 'sen. lisa murkowski', 'former president donald trump', 'one anti-cheney candidate', 'republican challenging rep. anthony gonzalez', 'house speaker nancy pelosi', 'new jersey golf club', 'new jersey', 'wyoming']
after google ner ['anti-trump rep. liz cheney', 'cheney challengers', 'rep. bennie thompson', 'rep. jamie raskin', 'wyoming state sen. anthony bouchard', 'sen. lisa murkowski', 'former president donald trump', 'one anti-cheney candidate', 'republican challenging rep. anthony gonzalez', 'house speaker nancy pelosi', 'new jersey golf club']
after similar words ['anti-trump rep. liz cheney', 'cheney challengers', 'rep. bennie thompson', 'rep. jamie raskin', 'wyoming state sen. anthony bouchard', 'sen. lisa murkowski', 'former president donald trump', 'one anti-cheney candidate', 'repub

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
# remove any empty keyphrase rows
df = df[df['keyphrases'].apply(lambda x: len(x)>0)]
df.keyphrases = df.keyphrases.astype(str)
df.drop("spacytextrank_keywords", axis=1, inplace=True)
df.drop("clean_body", axis=1, inplace=True)

In [17]:
df.to_csv("./mnet_test/new_keyphrases_last3.csv", index=False)

* Make datasets

In [2]:
train = pd.read_pickle("./data/preprocess_mnet_train.pkl")

train.drop_duplicates('natid', keep='first', inplace=True)
train.reset_index(drop=True, inplace=True)

print(train.shape)

(10078, 8)


In [3]:
test = pd.read_pickle("./data/preprocess_mnet_test.pkl")
test.drop_duplicates('natid', keep='first', inplace=True)
test.reset_index(drop=True, inplace=True)

print(test.shape)

(1258, 8)


In [4]:
full = pd.concat([train, test])
print("Before", full.shape)

full.drop_duplicates('natid', keep='first', inplace=True)
full.reset_index(drop=True, inplace=True)

print("After", full.shape)

sql= """
    SELECT 
        * 
    FROM 
        `api-project-901373404215.keyphrase_extraction.forbes_keyphrases` 
    """

# Send the query to the api and return a df
keyphrase_df = client.query(sql).to_dataframe()

print("Shape: ", keyphrase_df.shape) 

Before (11336, 8)
After (11046, 8)
Shape:  (77287, 4)


In [5]:
# articles with keyphrases already generated

bq_kps = pd.merge(full, 
                  keyphrase_df[['naturalid', 'keyphrases']], 
                  how="inner", 
                  left_on="natid", right_on="naturalid")
bq_kps.drop('naturalid', axis=1, inplace=True)

In [6]:
bq_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7,"[elon musks tesla, cryptocurrencies, private i..."


In [7]:
print(bq_kps.head(1)['keyphrases'][0])
print(bq_kps.tail(1)['keyphrases'][6742])

['elon musks tesla' 'cryptocurrencies' 'private investment' 'l ast week'
 'nasdaq listing' 'latest payment app' 'memberships'
 'mainstream acceptance' 'forbes) wework' 'usd coin' 'landlords'
 'spac deal' 'wework membership']
['prime day gaming laptop' 'best prime day chromebook deals chromebooks'
 'professional laptops' 'great deals' 'prime day laptop deals'
 'amazon prime day' 'best gaming laptops' 'acer chromebook'
 'best smartphones']


In [8]:
bq_kps.keyphrases = bq_kps.keyphrases.apply(lambda x: ', '.join(x))

In [10]:
bq_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7,"elon musks tesla, cryptocurrencies, private in..."


In [11]:
print(bq_kps.head(1)['keyphrases'][0])
print(bq_kps.tail(1)['keyphrases'][6742])

elon musks tesla, cryptocurrencies, private investment, l ast week, nasdaq listing, latest payment app, memberships, mainstream acceptance, forbes) wework, usd coin, landlords, spac deal, wework membership
prime day gaming laptop, best prime day chromebook deals chromebooks, professional laptops, great deals, prime day laptop deals, amazon prime day, best gaming laptops, acer chromebook, best smartphones


In [12]:
# articles with keyphrases newly generated above

new_kps = pd.read_csv("./mnet_test/new_keyphrases.csv")
print(new_kps.shape)
new_kps.duplicated().any()

(4301, 9)


False

In [14]:
new_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/3113-5f526f97559f59000...,2020-09-04,Music and Audio,Music and Audio,Hip Hop Music,Cardi B And Megan Thee Stallion Both Earn Thei...,cardi b megan thee stallion earn no. 1 hit you...,12,"['cardi bs forthcoming sophomore full-length',..."


In [15]:
print(new_kps.head(1)['keyphrases'][0])
print(new_kps.tail(1)['keyphrases'][4300])

['cardi bs forthcoming sophomore full-length', 'first appearance', 'cardi bs highest-charting cut', 'frames', 'hip-hop return', 'five consecutive turns', 'latest single', 'release date']
['new covid cases', 'positive cases', 'covid tracking project covid', 'case count', 'covid tracking project data', 'new coronavirus infections', 'highest number', 'hospitals', 'positive tests', 'massive upticks', ', new cases', 'masks', 'cold weather', 'national mask mandate']


In [17]:
import ast

new_kps.keyphrases = new_kps.keyphrases = new_kps.keyphrases.apply(ast.literal_eval).apply(lambda x: ', '.join(x))

In [19]:
new_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/3113-5f526f97559f59000...,2020-09-04,Music and Audio,Music and Audio,Hip Hop Music,Cardi B And Megan Thee Stallion Both Earn Thei...,cardi b megan thee stallion earn no. 1 hit you...,12,"cardi bs forthcoming sophomore full-length, fi..."


In [18]:
print(new_kps.head(1)['keyphrases'][0])
print(new_kps.tail(1)['keyphrases'][4300])

cardi bs forthcoming sophomore full-length, first appearance, cardi bs highest-charting cut, frames, hip-hop return, five consecutive turns, latest single, release date
new covid cases, positive cases, covid tracking project covid, case count, covid tracking project data, new coronavirus infections, highest number, hospitals, positive tests, massive upticks, , new cases, masks, cold weather, national mask mandate


In [20]:
# articles with keyphrases newly generated above

last3_new_kps = pd.read_csv("./mnet_test/new_keyphrases_last3.csv")
print(last3_new_kps.shape)
last3_new_kps.duplicated().any()

(3, 9)


False

In [21]:
last3_new_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/50276-60f734ce80505100...,2021-07-20,News and Politics,Sports,Golf,Trump Says Cheney Challengers Will Vie For His...,trump says cheney challengers vie endorsement ...,18,"['anti-trump rep. liz cheney', 'cheney challen..."


In [22]:
print(last3_new_kps.head(1)['keyphrases'][0])
print(last3_new_kps.tail(1)['keyphrases'][2])

['anti-trump rep. liz cheney', 'cheney challengers', 'rep. bennie thompson', 'rep. jamie raskin', 'wyoming state sen. anthony bouchard', 'sen. lisa murkowski', 'former president donald trump', 'one anti-cheney candidate', 'republican challenging rep. anthony gonzalez', 'house speaker nancy pelosi', 'new jersey golf club']
['target black friday deals', 'black friday couch deals rove concepts', 'allmoderns black friday sale', 'hayneedles black friday sale', 'ballards black friday sale', 'macys black friday couch deals macys', 'pottery barns black friday sale']


In [23]:
last3_new_kps.keyphrases = last3_new_kps.keyphrases.apply(ast.literal_eval).apply(lambda x: ', '.join(x))

In [24]:
last3_new_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/50276-60f734ce80505100...,2021-07-20,News and Politics,Sports,Golf,Trump Says Cheney Challengers Will Vie For His...,trump says cheney challengers vie endorsement ...,18,"anti-trump rep. liz cheney, cheney challengers..."


In [25]:
print(last3_new_kps.head(1)['keyphrases'][0])
print(last3_new_kps.tail(1)['keyphrases'][2])

anti-trump rep. liz cheney, cheney challengers, rep. bennie thompson, rep. jamie raskin, wyoming state sen. anthony bouchard, sen. lisa murkowski, former president donald trump, one anti-cheney candidate, republican challenging rep. anthony gonzalez, house speaker nancy pelosi, new jersey golf club
target black friday deals, black friday couch deals rove concepts, allmoderns black friday sale, hayneedles black friday sale, ballards black friday sale, macys black friday couch deals macys, pottery barns black friday sale


In [26]:
all_kps = pd.concat([bq_kps, new_kps, last3_new_kps])
all_kps.drop_duplicates('natid', keep='first', inplace=True)
all_kps.reset_index(drop=True, inplace=True)
all_kps.shape

(11046, 9)

In [27]:
train[train.natid.isin(all_kps.natid)].shape

(10078, 8)

In [28]:
test[~test.natid.isin(all_kps.natid)].shape

(0, 8)

In [29]:
all_kps.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,keyphrases
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7,"elon musks tesla, cryptocurrencies, private in..."


In [32]:
train_kps = pd.merge(train, 
                     all_kps[['natid', 'keyphrases']], 
                     how='left', 
                     on='natid')
print(train_kps.shape)
# print(train_kps.index)

(10078, 9)


In [33]:
test_kps = pd.merge(test, 
                     all_kps[['natid', 'keyphrases']], 
                     how='left', 
                     on='natid')
print(test_kps.shape)

(1258, 9)


In [35]:
train_kps.to_csv("./data/mnet_train_keyphrase.csv", index=False)

train_kps.to_pickle("./data/mnet_train_keyphrase.pkl")

In [36]:
test_kps.to_csv("./data/mnet_test_keyphrase.csv", index=False)

test_kps.to_pickle("./data/mnet_test_keyphrase.pkl")

* Testing files to make sure no errors

In [37]:
train_csv = pd.read_csv("./data/mnet_train_keyphrase.csv")
print(train_csv.shape)
print(train_csv.duplicated().any())

(10078, 9)
False


In [38]:
train_csv.tail(1)['keyphrases'][10077]

'college degrees, new jobs, upward mobility opportunities, college education, college proposition, upward jump, increasing opportunities, economic research, surveyed employers, stale researchers, new working paper, better benefits, few compelling data, qualified applicants, lingering doubts, little trouble, new report, employment and wage data, new paper'

In [39]:
train_pkl = pd.read_pickle("./data/mnet_train_keyphrase.pkl")
print(train_pkl.shape)
print(train_pkl.duplicated('natid').any())

(10078, 9)
False


In [40]:
train_pkl.tail(1)['keyphrases'][10077]

'college degrees, new jobs, upward mobility opportunities, college education, college proposition, upward jump, increasing opportunities, economic research, surveyed employers, stale researchers, new working paper, better benefits, few compelling data, qualified applicants, lingering doubts, little trouble, new report, employment and wage data, new paper'

In [41]:
print(train_csv[train_csv.natid == train_pkl.natid].shape)
print(train_csv[train_csv.pub_date == train_pkl.pub_date].shape)
print(train_csv[train_csv.category == train_pkl.category].shape)
print(train_csv[train_csv.tier1 == train_pkl.tier1].shape)
print(train_csv[train_csv.tier2 == train_pkl.tier2].shape)
print(train_csv[train_csv.summary == train_pkl.summary].shape)
print(train_csv[train_csv.clean_summary == train_pkl.clean_summary].shape)
print(train_csv[train_csv.label == train_pkl.label].shape)
print(train_csv[train_csv.keyphrases == train_pkl.label].keyphrases)

print(train_csv.pub_date[0])
print(train_pkl.pub_date[0])

print(train_csv.pub_date[0] == train_pkl.pub_date[0])

print(train_csv.keyphrases[0])
print(train_pkl.keyphrases[0])

print(train_csv.keyphrases[0] == train_pkl.keyphrases[0])

(10078, 9)
(0, 9)
(10078, 9)
(10078, 9)
(10078, 9)
(10078, 9)
(10078, 9)
(10078, 9)
Series([], Name: keyphrases, dtype: object)
2021-04-20
2021-04-20
False
elon musks tesla, cryptocurrencies, private investment, l ast week, nasdaq listing, latest payment app, memberships, mainstream acceptance, forbes) wework, usd coin, landlords, spac deal, wework membership
elon musks tesla, cryptocurrencies, private investment, l ast week, nasdaq listing, latest payment app, memberships, mainstream acceptance, forbes) wework, usd coin, landlords, spac deal, wework membership
True
